In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
os.chdir('../../../')

In [3]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import *
from musicautobot.utils.midifile import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *

In [4]:
import traceback
import time

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [5]:
version = 'v20'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
import pandas as pd

In [7]:
out_dir = 's2s_encode'

In [8]:
source_dir = 'midi_sources'
source_csv = version_path/'metadata'/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv, out_csv

(PosixPath('data/midi/v20/metadata/midi_sources.csv'),
 PosixPath('data/midi/v20/s2s_encode/s2s_encode.csv'))

In [9]:
# num_comps = 2 # note, duration
cutoff = 5 # max instruments
min_variation = 3 # minimum number of different midi notes played
# max_dur = 128

### Encoding midi to numpy

In [10]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/musicautobot/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (3,4,6,7,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ht_time_signature,genres,midi,midi_title,parts,ht_bpm,mxl,section,source,artist,md5,song_url,ht_key,title,ht_mode
0,4.0,NaN,midi_sources/hooktheory/pianoroll/w/wayne-shar...,yu-gi-oh3,"intro,chorus",128.0,NaN,chorus,hooktheory,wayne-sharpe,bf1f29e5ff84e3e93e37fb873bfb590e,https://www.hooktheory.com/theorytab/view/wayn...,C,yu-gi-oh-theme-song,1.0
1,3.0,NaN,midi_sources/hooktheory/pianoroll/w/wayne-shar...,yu-gi-oh,"intro,chorus",85.0,NaN,intro,hooktheory,wayne-sharpe,055f80ad67f64edb14a85ca8fbfe8c29,https://www.hooktheory.com/theorytab/view/wayn...,C,yu-gi-oh-theme-song,1.0
2,4.0,Jazz,midi_sources/hooktheory/pianoroll/w/what-a-day...,kiefer,chorus,96.0,NaN,chorus,hooktheory,what-a-day,197f96f5d181f6ce1e2c5ab04ac1ff87,https://www.hooktheory.com/theorytab/view/what...,D,kiefer,6.0
3,4.0,"J-Pop,Pop",midi_sources/hooktheory/pianoroll/w/whiteflame...,senbonzakura - pre-Pre-Chorus,"verse,pre-chorus,chorus",152.0,NaN,pre-chorus,hooktheory,whiteflame,9e7ce13a35f1314423a9a6d5a5287a4a,https://www.hooktheory.com/theorytab/view/whit...,D,senbonzakura,6.0
4,4.0,"J-Pop,Pop",midi_sources/hooktheory/pianoroll/w/whiteflame...,Senbonzakura,"verse,pre-chorus,chorus",152.0,NaN,verse,hooktheory,whiteflame,d5aaf79d0989222f1362f9f46c540a27,https://www.hooktheory.com/theorytab/view/whit...,D,senbonzakura,6.0


In [11]:
all_records = df.to_dict(orient='records'); len(all_records)

197426

In [12]:
def process_metadata(metadata):
    result = metadata.copy()
    
    # Part 1. Compress tracks/instruments
    if not isinstance(metadata.get('midi'), str): return None
    
    input_path = version_path/metadata['midi']
    extension = input_path.suffix.lower()
    if not input_path.exists(): 
        print('Input path does not exist:', input_path, metadata)
        return result
    
    # Get outfile and check if it exists
    out_file = Path(str(input_path).replace(f'/{source_dir}/', f'/{out_dir}/'))
    out_file = out_file.with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): 
        result['numpy'] = str(out_file.relative_to(version_path))
        return result
    
    npenc = transform_midi(input_path)
    if npenc is None: return result
    np.save(out_file, npenc)
    result['numpy'] = str(out_file.relative_to(version_path))
    return result

In [13]:
def transform_midi(midi_file):
    input_path = midi_file
    
    try: 
        if num_piano_tracks(input_path) not in [1, 2]: return None
        input_file = compress_midi_file(input_path, min_variation=min_variation, cutoff=cutoff) # remove non note tracks and standardize instruments
        if not input_file: return None
    except Exception as e:
        if 'badly form' in str(e): return None # ignore badly formatted midi errors
        if 'out of range' in str(e): return None # ignore badly formatted midi errors
        print('Error parsing midi', input_path, e)
        return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        print('Could not encode to chordarr:', input_path, e)
#         print(traceback.format_exc())
        return None
    
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
#     if delta_trim > 300: 
#         print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
#         return None
    chordarr = chord_short
    
    # Only 2 piano parts allowed
    _,num_parts,_ = chordarr.shape
    if num_parts != 2: return None
    
    # Individual parts must have notes
    parts = [part_enc(chordarr, i) for i in range(num_parts)]
    for p in parts: 
        if not is_valid_npenc(p, min_notes=8, input_path=input_path): return None
        
    # order by melody > chords
    p1, p2 = parts
    m, c = (p1, p2) if avg_pitch(p1) > avg_pitch(p2) else (p2, p1) # Assuming melody has higher pitch
    
    return np.array([m, c])

In [14]:
# transform_midi(piano_file)
midi_mxl_file = version_path/'midi_sources/from_mxl/musescore/data/49143.mid'
input_file = midi_mxl_file
stream = file2stream(input_file) # 1.
chordarr = stream2chordarr(stream)

In [15]:
chordarr.shape

(1021, 2, 128)

In [16]:
transform_midi(midi_mxl_file).shape

(2,)

In [17]:
def try_process_metadata(metadata):
    try:
        return process_metadata(metadata)
    except Exception:
#         print(traceback.format_exc())
        return None

In [18]:
# # sanity check
import random
for r in random.sample(all_records, 10):
    process_metadata(r)

In [19]:
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

In [ ]:
processed = process_all(try_process_metadata, all_records, timeout=300, timeout_func=timeout_func)

Sequence too short: 6 data/midi/v20/midi_sources/hooktheory/pianoroll/w/willie-nelson/you-were-always-on-my-mind/verse_key_original.mid


Sequence too short: 4 data/midi/v20/midi_sources/hooktheory/pianoroll/y/yasunori-mitsuda/black-omen/instrumental_key_original.mid
Sequence too short: 3 data/midi/v20/midi_sources/hooktheory/pianoroll/y/yes/heart-of-the-sunrise/intro_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/y/yo-la-tengo/ohm/verse_key_original.mid
Sequence too short: 5 data/midi/v20/midi_sources/hooktheory/pianoroll/j/janet-jackson/nasty/intro_key_original.mid
Sequence too short: 3 data/midi/v20/midi_sources/hooktheory/pianoroll/j/jose-gonzales/step-out/intro_key_original.mid
Sequence too short: 3 data/midi/v20/midi_sources/hooktheory/pianoroll/j/john-tesh/roundball-rock/intro_key_original.mid
Sequence too short: 3 data/midi/v20/midi_sources/hooktheory/pianoroll/j/john-powell/how-to-train-your-dragon---test-drive/intro_key_original.mid
Sequence too short: 3 data/midi/v20/midi_sources/hooktheory/pianoroll/j/jay-hardway---mike-hawkins/freedom/chorus_key_original.mid
Sequence t

Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/s/steve-jablonsky/lone-survivor/verse_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/c/coldplay/hurts-like-heaven/solo_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/c/c418/acid/verse_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/c/childish-gambino/me-and-your-mama/intro_key_original.mid
Sequence too short: 4 data/midi/v20/midi_sources/hooktheory/pianoroll/c/creedence-clearwater-revival/born-on-the-bayou/intro_key_original.mid
Sequence too short: 3 data/midi/v20/midi_sources/hooktheory/pianoroll/c/cysmix/aumetra-the-witch/intro_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/c/capcom/okami---yami/intro-and-verse_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/c/clarian/castaway---henry-saiz-remix/chorus_key_original.mid
n

Sequence too short: 4 data/midi/v20/midi_sources/hooktheory/pianoroll/t/the-beatles/you-cant-do-that/intro_key_original.mid
Sequence too short: 4 data/midi/v20/midi_sources/hooktheory/pianoroll/t/the-beatles/ticket-to-ride/outro_key_original.mid
Sequence too short: 4 data/midi/v20/midi_sources/hooktheory/pianoroll/t/the-lorax/let-it-grow/intro_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/o/outkast/hey-ya/chorus_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/o/odd-future/analog-2/instrumental_key_original.mid
Sequence too short: 5 data/midi/v20/midi_sources/hooktheory/pianoroll/k/koji-kondo/delfino-plaza/intro_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/k/koji-kondo/super-mario-bros-3---hammer-bros-battle-theme/intro_key_original.mid
Sequence too short: 7 data/midi/v20/midi_sources/hooktheory/pianoroll/k/king-gizzard-and-the-lizard-wizard/gamma-knife/verse_key_ori

In [ ]:
arr2csv(processed, out_csv); len(processed)

In [ ]:
df = pd.read_csv(out_csv); df.head()

In [ ]:
len([f for f in df.numpy.values if isinstance(f, str)])

In [ ]:
old 51580

In [34]:
from collections import Counter

In [35]:
df[df.numpy.notnull()]

,ht_mode,title,genres,source,section,midi,artist,parts,ht_bpm,midi_title,ht_time_signature,ht_key,ht_offset,md5,song_url,numpy,mxl
0,1.0,yu-gi-oh-theme-song,NaN,hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/wayne-shar...,wayne-sharpe,"intro,chorus",128.0,yu-gi-oh3,4.0,C,0.0,bf1f29e5ff84e3e93e37fb873bfb590e,https://www.hooktheory.com/theorytab/view/wayn...,sf4/s2s_encode/hooktheory/pianoroll/w/wayne-sh...,NaN
2,6.0,kiefer,Jazz,hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/what-a-day...,what-a-day,chorus,96.0,kiefer,4.0,D,-5.0,197f96f5d181f6ce1e2c5ab04ac1ff87,https://www.hooktheory.com/theorytab/view/what...,sf4/s2s_encode/hooktheory/pianoroll/w/what-a-d...,NaN
3,6.0,senbonzakura,"J-Pop,Pop",hooktheory,pre-chorus,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,"verse,pre-chorus,chorus",152.0,senbonzakura - pre-Pre-Chorus,4.0,D,-5.0,9e7ce13a35f1314423a9a6d5a5287a4a,https://www.hooktheory.com/theorytab/view/whit...,sf4/s2s_encode/hooktheory/pianoroll/w/whitefla...,NaN
4,6.0,senbonzakura,"J-Pop,Pop",hooktheory,verse,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,"verse,pre-chorus,chorus",152.0,Senbonzakura,4.0,D,-5.0,d5aaf79d0989222f1362f9f46c540a27,https://www.hooktheory.com/theorytab/view/whit...,sf4/s2s_encode/hooktheory/pianoroll/w/whitefla...,NaN
5,6.0,senbonzakura,"J-Pop,Pop",hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,"verse,pre-chorus,chorus",152.0,Senbonzakura,4.0,D,-5.0,e0c189ee753b30c4758d85211f13c189,https://www.hooktheory.com/theorytab/view/whit...,sf4/s2s_encode/hooktheory/pianoroll/w/whitefla...,NaN
6,1.0,last-christmas,Holiday,hooktheory,verse,midi_sources/hooktheory/pianoroll/w/wham/last-...,wham,"intro,verse,chorus",108.0,Last Christmas Verse,4.0,Db,-1.0,38e38402443506e326b76536e8e327a0,https://www.hooktheory.com/theorytab/view/wham...,sf4/s2s_encode/hooktheory/pianoroll/w/wham/las...,NaN
7,1.0,last-christmas,Holiday,hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/wham/last-...,wham,"intro,verse,chorus",108.0,Last Christmas Chorus,4.0,Db,-1.0,75d0251177c8c1fa9a02821299fa5ba8,https://www.hooktheory.com/theorytab/view/wham...,sf4/s2s_encode/hooktheory/pianoroll/w/wham/las...,NaN
8,1.0,last-christmas,Holiday,hooktheory,intro,midi_sources/hooktheory/pianoroll/w/wham/last-...,wham,"intro,verse,chorus",108.0,Last Christmas Intro,4.0,Db,-1.0,83d2a800f40aeca07e30e4718cda8fe5,https://www.hooktheory.com/theorytab/view/wham...,sf4/s2s_encode/hooktheory/pianoroll/w/wham/las...,NaN
9,1.0,freedom,NaN,hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/wham/freed...,wham,chorus,128.0,Freedom Chorus,4.0,C,0.0,60fa29cfec107df27b053cf9708823d5,https://www.hooktheory.com/theorytab/view/wham...,sf4/s2s_encode/hooktheory/pianoroll/w/wham/fre...,NaN
11,1.0,west-wing-suite,NaN,hooktheory,instrumental,midi_sources/hooktheory/pianoroll/w/wg-snuffy-...,wg-snuffy-walden,instrumental,86.0,snuffy,4.0,G,5.0,a856dff6c54398544c217104d047abe0,https://www.hooktheory.com/theorytab/view/wg-s...,sf4/s2s_encode/hooktheory/pianoroll/w/wg-snuff...,NaN


In [36]:
Counter(df[df.numpy.notnull()].source.values)

Counter({'hooktheory': 17700,
         'freemidi': 51,
         'midiworld': 66,
         'ecomp': 2357,
         'cprato': 124,
         'classical_piano': 319,
         'classical_archives': 2399,
         'musescore': 6908,
         'wikifonia': 32,
         'lmd': 128,
         'reddit': 5266,
         'hooktheory_c': 17830})

In [37]:
len(df[df.numpy.notnull()].source.values)

53180

In [38]:
Counter(df[df.numpy.notnull()].source.values)

Counter({'hooktheory': 17700,
         'freemidi': 51,
         'midiworld': 66,
         'ecomp': 2357,
         'cprato': 124,
         'classical_piano': 319,
         'classical_archives': 2399,
         'musescore': 6908,
         'wikifonia': 32,
         'lmd': 128,
         'reddit': 5266,
         'hooktheory_c': 17830})

In [29]:
Counter(df.source.values)

Counter({'hooktheory': 19882,
         'freemidi': 5166,
         'midiworld': 4107,
         'ecomp': 2507,
         'cprato': 312,
         'classical_piano': 329,
         'classical_archives': 14641,
         'musescore': 10928,
         'wikifonia': 6346,
         'lmd': 13565,
         'reddit': 98659,
         'hooktheory_c': 20076})

## Convert to hooktheory databunch

In [10]:
out_path = version_path/out_dir

In [11]:
csv = pd.read_csv(out_csv); csv.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,7,9,11,14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ht_mode,title,genres,source,section,midi,artist,parts,ht_bpm,midi_title,ht_time_signature,ht_key,ht_offset,md5,song_url,numpy,mxl
0,1.0,yu-gi-oh-theme-song,NaN,hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/wayne-shar...,wayne-sharpe,"intro,chorus",128.0,yu-gi-oh3,4.0,C,0.0,bf1f29e5ff84e3e93e37fb873bfb590e,https://www.hooktheory.com/theorytab/view/wayn...,sf4/s2s_encode/hooktheory/pianoroll/w/wayne-sh...,NaN
1,1.0,yu-gi-oh-theme-song,NaN,hooktheory,intro,midi_sources/hooktheory/pianoroll/w/wayne-shar...,wayne-sharpe,"intro,chorus",85.0,yu-gi-oh,3.0,C,0.0,055f80ad67f64edb14a85ca8fbfe8c29,https://www.hooktheory.com/theorytab/view/wayn...,NaN,NaN
2,6.0,kiefer,Jazz,hooktheory,chorus,midi_sources/hooktheory/pianoroll/w/what-a-day...,what-a-day,chorus,96.0,kiefer,4.0,D,-5.0,197f96f5d181f6ce1e2c5ab04ac1ff87,https://www.hooktheory.com/theorytab/view/what...,sf4/s2s_encode/hooktheory/pianoroll/w/what-a-d...,NaN
3,6.0,senbonzakura,"J-Pop,Pop",hooktheory,pre-chorus,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,"verse,pre-chorus,chorus",152.0,senbonzakura - pre-Pre-Chorus,4.0,D,-5.0,9e7ce13a35f1314423a9a6d5a5287a4a,https://www.hooktheory.com/theorytab/view/whit...,sf4/s2s_encode/hooktheory/pianoroll/w/whitefla...,NaN
4,6.0,senbonzakura,"J-Pop,Pop",hooktheory,verse,midi_sources/hooktheory/pianoroll/w/whiteflame...,whiteflame,"verse,pre-chorus,chorus",152.0,Senbonzakura,4.0,D,-5.0,d5aaf79d0989222f1362f9f46c540a27,https://www.hooktheory.com/theorytab/view/whit...,sf4/s2s_encode/hooktheory/pianoroll/w/whitefla...,NaN


In [12]:
csv['numpy'] = csv['numpy'].apply(lambda x: x.replace('sf4/', '') if isinstance(x, str) else x)
csv.to_csv(out_csv, index=False)

In [13]:
def get_files(csv):
    files = csv['numpy']
    flist = [Path(version_path/f) for f in files.values if isinstance(f, str)]
    flist = [f for f in flist if f.exists()]
    return flist

In [14]:
def create_databunch(files, data_save_name, path=out_path):
    save_file = path/data_save_name
    if save_file.exists():
        data = load_data(path, data_save_name)
    else:
        save_file.parent.mkdir(exist_ok=True, parents=True)
        vocab = MusicVocab.create()
        processors = [S2SFileProcessor(), S2SPartsProcessor()]

        data = MusicDataBunch.from_files(files, path, processors=processors, 
                                          preloader_cls=S2SPreloader, list_cls=S2SItemList)
        data.save(data_save_name)
    return data

In [15]:
hook_csv = csv.loc[csv.source.isin(['hooktheory'])]
hook_files = get_files(hook_csv); len(hook_files)
hook_data = create_databunch(hook_files, 'cached/hook.pkl')

In [16]:
hook_csv = csv.loc[csv.source.isin(['hooktheory_c'])]
hook_files = get_files(hook_csv); len(hook_files)
hook_data = create_databunch(hook_files, 'cached/hook_c.pkl')

In [17]:
lq_csv = csv.loc[csv.source.isin(['reddit', 'classical_piano', 'ecomp', 'midiworld', 'freemidi', 'lmd', 'cprato', 'wikifonia', 'classical_archives'])]
lq_files = get_files(lq_csv); len(lq_files)
lq_data = create_databunch(lq_files, 'cached/lq.pkl')

In [18]:
hq_csv = csv.loc[csv.source.isin(['hooktheory', 'musescore'])]
hq_files = get_files(hq_csv); len(hq_files)
hq_data = create_databunch(hq_files, 'cached/hq.pkl')

In [19]:
len(hook_data.train_dl.dl.dataset)

15173

In [20]:
all_files = get_files(csv); len(all_files)
all_data = create_databunch(all_files, 'cached/all.pkl')

In [21]:
import random
sample_data = create_databunch(random.sample(all_files, 1000), 'cached/sample.pkl')

## Load data

In [41]:
single_tfm = partial(to_single_stream, vocab=vocab)
load_data =  MusicDataBunch.load(path=out_path, cache_name='tmp/hook_c', preloader_cls=S2SPreloader, train_tfms=[single_tfm])

You can deactivate this warning by passing `no_check=True`.


/home/ubuntu/fastai/fastai/basic_data.py:259: UserWarning: There seems to be something wrong with your dataset, for example, in the first batch can't access any element of self.train_ds.
Tried: 0,1,2,3,4...
  warn(warn_msg)


In [42]:
load_data =  MusicDataBunch.load(path=out_path, cache_name='tmp/hook_c', preloader_cls=S2SPreloader)

In [49]:
load_data.one_batch()

(tensor([[  6, 622,  88,  ..., 143,   8, 143],
         [  5, 622,  55,  ..., 161,  51, 161],
         [  5, 622,  65,  ..., 149,  61, 149],
         ...,
         [  5, 622,  66,  ..., 185,  75, 185],
         [  5, 622,  64,  ..., 173,  61, 173],
         [  5, 622,  63,  ..., 185,  60, 185]]),
 tensor([[  3,   5, 622,  ..., 167,  78, 155],
         [  3,   6, 622,  ..., 143,   8, 143],
         [  3,   6, 622,  ..., 140,   8, 140],
         ...,
         [  3,   6, 622,  ..., 143,  87, 143],
         [  3,   6, 622,  ..., 149,   8, 149],
         [  3,   6, 622,  ..., 149,  75, 197]]))

In [ ]:
# ps = [S2SFileProcessor()]

# single_tfm = partial(to_single_stream, vocab=vocab)
# data = (MusicItemList(items=hook_files[:100], path=out_path, processor=ps, tfms=[single_tfm])
#         .split_by_rand_pct(0.01, seed=6)
#         .label_const(label_cls=LMLabelList))
# data.x._bunch = MusicDataBunch

In [78]:
# data.x.tfms = [single_tfm]

In [ ]:
data = data.databunch(bs=4, preloader_cls=S2SPreloader, train_tfms=[single_tfm])

In [80]:
out = data.train_dl.dl.dataset[0]

In [ ]:
data.one_batch()